# Migration from the csv database to sql of the registration matrixes

We are going to load the csv database, and load its information into the sql database automatically.

## Imports

In [ ]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import queries.queries as qrs
from pathlib import Path
import tifffile as tiff
import pandas as pd
from tqdm import tqdm

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

## load the csv database

In [ ]:
df = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

df = df.replace({np.nan: None})

#get only the rows where 'Id' contains JI
df = df[df['Id'].str.contains('JI')]

df.head(5)

## Migrate

Now we just have to iterate the dataframe, retrieve the data and load it into the database

### Load needed tables

Some tables of the database have to be loaded to get some index info to load the registrations, we load them now so no overhead is acumulated

In [ ]:
measurements = qrs.get_data('measurements')

measurements.head(5)

In [ ]:
#table name to load
table_name = 'measurement_registrations'

description = '2024 registration methodology.'

#iterate over the rows of the dataframe

for index, row in tqdm(df.iterrows(), total=df.shape[0]):

    xct_file = row['Frontal 90 Right']

    ut_file = row['UT']

    registration_matrix = row['Registration Parameters']

    if xct_file is None or ut_file is None:
        print(f"Skipping row {index} because it has no XCT or UT file")
        continue

    if registration_matrix is None:
        print(f"Skipping row {index} because it has no registration matrix")
        continue

    #get the id of the ut measurement
    ut_id = measurements[measurements['file_path_measurement'] == str(ut_file)]['id_measurement'].values[0]

    #get the id of the xct measurement
    xct_id = measurements[measurements['file_path_measurement'] == str(xct_file)]['id_measurement'].values[0]

    main_parameters = {'reference_measurement_id':ut_id,'registered_measurement_id':xct_id,'registration_matrix':registration_matrix,'description':description}

    try:

        # Extract column names and values from the attributes dictionary
        columns = ', '.join(main_parameters.keys())
        values = ', '.join([f"'{v}'" for v in main_parameters.values()])

        # Construct the SQL INSERT statement
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

        # Create a cursor object using the cursor() method
        cursor = conn.cursor()

        # Execute the SQL statement
        cursor.execute(sql)

        cursor.execute('COMMIT')

        cursor.close()
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error inserting row {index}: {error}")
